# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np


Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)

print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
import torch
torch.cuda.is_available()

In [5]:
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from dqn_agent import Agent


agent = Agent(state_size=37, action_size=4, seed=0)
eps_start = 1.0
eps_end=0.01
eps_decay=0.995
n_episodes = 5000
max_t = 10000

In [9]:
scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores
eps = eps_start                    # initialize epsilon
for i_episode in range(1, n_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name] # reset the environment
    state = env_info.vector_observations[0] 
    score = 0
    for t in range(max_t):
        action = agent.act(state, eps)
        env_info = env.step(action.astype(np.int32))[brain_name]  
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)),end="")
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 2.97
Episode 100	Average Score: 2.94
Episode 100	Average Score: 2.91
Episode 100	Average Score: 2.88
Episode 100	Average Score: 2.85
Episode 100	Average Score: 2.82
Episode 100	Average Score: 2.79
Episode 100	Average Score: 2.76
Episode 100	Average Score: 2.73
Episode 100	Average Score: 2.70
Episode 100	Average Score: 2.67
Episode 100	Average Score: 2.64
Episode 100	Average Score: 2.61
Episode 100	Average Score: 2.58
Episode 100	Average Score: 2.55
Episode 100	Average Score: 2.52
Episode 100	Average Score: 2.49
Episode 100	Average Score: 2.46
Episode 100	Average Score: 2.43
Episode 100	Average Score: 2.40
Episode 100	Average Score: 2.37
Episode 100	Average Score: 2.34
Episode 100	Average Score: 2.31
Episode 100	Average Score: 2.28
Episode 100	Average Score: 2.26
Episode 100	Average Score: 2.24
Episode 100	Average Score: 2.22
Episode 100	Average Score: 2.21
Episode 100	Average Score: 2.20
Episode 100	Average Score: 2.19
Episode 100	Average Score: 2.18
Episode 

Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 100	Average Score: 5.00
Episode 200	Average Score: 7.134
Episode 200	Average Score: 7.07
Episode 200	Average Score: 7.01
Episode 200	Average Score: 6.95
Episode 200	Average Score: 6.89
Episode 200	Average Score: 6.82
Episode 200	Average Score: 6.75
Episode 200	Average Score: 6.68
Episode 200	Average Score: 6.61
Episode 200	Average Score: 6.54
Episode

Episode 200	Average Score: 10.85
Episode 200	Average Score: 10.93
Episode 200	Average Score: 11.01
Episode 200	Average Score: 11.08
Episode 200	Average Score: 11.15
Episode 200	Average Score: 11.22
Episode 200	Average Score: 11.29
Episode 200	Average Score: 11.36
Episode 200	Average Score: 11.43
Episode 200	Average Score: 11.50
Episode 200	Average Score: 11.57
Episode 200	Average Score: 11.64
Episode 200	Average Score: 11.71
Episode 200	Average Score: 11.78
Episode 200	Average Score: 11.85
Episode 300	Average Score: 13.64
Episode 300	Average Score: 13.52
Episode 300	Average Score: 13.40
Episode 300	Average Score: 13.28
Episode 300	Average Score: 13.16
Episode 300	Average Score: 13.05
Episode 300	Average Score: 12.94
Episode 300	Average Score: 12.83
Episode 300	Average Score: 12.72
Episode 300	Average Score: 12.61
Episode 300	Average Score: 12.50
Episode 300	Average Score: 12.39
Episode 300	Average Score: 12.28
Episode 300	Average Score: 12.17
Episode 300	Average Score: 12.06
Episode 30

Episode 300	Average Score: 13.19
Episode 300	Average Score: 13.25
Episode 300	Average Score: 13.31
Episode 300	Average Score: 13.37
Episode 300	Average Score: 13.43
Episode 400	Average Score: 11.47
Episode 400	Average Score: 11.36
Episode 400	Average Score: 11.25
Episode 400	Average Score: 11.14
Episode 400	Average Score: 11.03
Episode 400	Average Score: 10.92
Episode 400	Average Score: 10.81
Episode 400	Average Score: 10.70
Episode 400	Average Score: 10.59
Episode 400	Average Score: 10.48
Episode 400	Average Score: 10.37
Episode 400	Average Score: 10.26
Episode 400	Average Score: 10.15
Episode 400	Average Score: 10.04
Episode 400	Average Score: 9.93
Episode 400	Average Score: 9.82
Episode 400	Average Score: 9.71
Episode 400	Average Score: 9.60
Episode 400	Average Score: 9.49
Episode 400	Average Score: 9.38
Episode 400	Average Score: 9.27
Episode 400	Average Score: 9.16
Episode 400	Average Score: 9.05
Episode 400	Average Score: 8.94
Episode 400	Average Score: 8.83
Episode 400	Average S

Episode 400	Average Score: 11.21
Episode 400	Average Score: 11.26
Episode 400	Average Score: 11.31
Episode 400	Average Score: 11.36
Episode 400	Average Score: 11.41
Episode 400	Average Score: 11.46
Episode 400	Average Score: 11.51
Episode 400	Average Score: 11.56
Episode 400	Average Score: 11.61
Episode 400	Average Score: 11.66
Episode 400	Average Score: 11.71
Episode 400	Average Score: 11.76
Episode 400	Average Score: 11.81
Episode 400	Average Score: 11.86
Episode 400	Average Score: 11.91
Episode 400	Average Score: 11.96
Episode 400	Average Score: 12.02
Episode 400	Average Score: 12.08
Episode 400	Average Score: 12.14
Episode 400	Average Score: 12.20
Episode 400	Average Score: 12.26
Episode 400	Average Score: 12.32
Episode 400	Average Score: 12.38
Episode 400	Average Score: 12.44
Episode 400	Average Score: 12.50
Episode 400	Average Score: 12.56
Episode 400	Average Score: 12.62
Episode 500	Average Score: 14.73
Episode 500	Average Score: 14.62
Episode 500	Average Score: 14.51
Episode 50

...

Episode 1700	Average Score: 8.74
Episode 1700	Average Score: 8.78
Episode 1700	Average Score: 8.82
Episode 1700	Average Score: 8.86
Episode 1700	Average Score: 8.90
Episode 1700	Average Score: 8.94
Episode 1700	Average Score: 8.98
Episode 1700	Average Score: 9.02
Episode 1700	Average Score: 9.06
Episode 1700	Average Score: 9.09
Episode 1700	Average Score: 9.12
Episode 1700	Average Score: 9.15
Episode 1700	Average Score: 9.18
Episode 1700	Average Score: 9.21
Episode 1700	Average Score: 9.24
Episode 1700	Average Score: 9.27
Episode 1700	Average Score: 9.30
Episode 1700	Average Score: 9.33
Episode 1700	Average Score: 9.36
Episode 1700	Average Score: 9.39
Episode 1700	Average Score: 9.41
Episode 1700	Average Score: 9.43
Episode 1700	Average Score: 9.45
Episode 1700	Average Score: 9.47
Episode 1700	Average Score: 9.49
Episode 1700	Average Score: 9.51
Episode 1700	Average Score: 9.53
Episode 1700	Average Score: 9.55
Episode 1700	Average Score: 9.57
Episode 1700	Average Score: 9.59
Episode 17

Episode 1800	Average Score: 12.68
Episode 1800	Average Score: 12.76
Episode 1800	Average Score: 12.84
Episode 1800	Average Score: 12.92
Episode 1800	Average Score: 13.00
Episode 1800	Average Score: 13.08
Episode 1800	Average Score: 13.16
Episode 1800	Average Score: 13.24
Episode 1800	Average Score: 13.32
Episode 1800	Average Score: 13.40
Episode 1800	Average Score: 13.48
Episode 1800	Average Score: 13.56
Episode 1800	Average Score: 13.64
Episode 1800	Average Score: 13.72
Episode 1800	Average Score: 13.80
Episode 1800	Average Score: 13.88
Episode 1800	Average Score: 13.97
Episode 1800	Average Score: 14.06
Episode 1800	Average Score: 14.14
Episode 1800	Average Score: 14.22
Episode 1800	Average Score: 14.30
Episode 1800	Average Score: 14.38
Episode 1800	Average Score: 14.46
Episode 1800	Average Score: 14.54
Episode 1800	Average Score: 14.62
Episode 1800	Average Score: 14.70
Episode 1800	Average Score: 14.78
Episode 1800	Average Score: 14.86
Episode 1800	Average Score: 14.94
Episode 1800	A

Episode 1900	Average Score: 7.46
Episode 1900	Average Score: 7.53
Episode 1900	Average Score: 7.60
Episode 1900	Average Score: 7.67
Episode 1900	Average Score: 7.74
Episode 1900	Average Score: 7.81
Episode 1900	Average Score: 7.88
Episode 1900	Average Score: 7.95
Episode 1900	Average Score: 8.02
Episode 1900	Average Score: 8.09
Episode 1900	Average Score: 8.16
Episode 1900	Average Score: 8.23
Episode 1900	Average Score: 8.30
Episode 1900	Average Score: 8.37
Episode 1900	Average Score: 8.44
Episode 1900	Average Score: 8.50
Episode 1900	Average Score: 8.56
Episode 1900	Average Score: 8.62
Episode 1900	Average Score: 8.68
Episode 1900	Average Score: 8.74
Episode 1900	Average Score: 8.80
Episode 1900	Average Score: 8.86
Episode 1900	Average Score: 8.92
Episode 1900	Average Score: 8.98
Episode 1900	Average Score: 9.04
Episode 1900	Average Score: 9.10
Episode 1900	Average Score: 9.16
Episode 1900	Average Score: 9.22
Episode 1900	Average Score: 9.28
Episode 1900	Average Score: 9.34
Episode 19

Episode 2000	Average Score: 8.16
Episode 2000	Average Score: 8.24
Episode 2000	Average Score: 8.32
Episode 2000	Average Score: 8.40
Episode 2000	Average Score: 8.48
Episode 2000	Average Score: 8.56
Episode 2000	Average Score: 8.64
Episode 2000	Average Score: 8.72
Episode 2000	Average Score: 8.79
Episode 2000	Average Score: 8.86
Episode 2000	Average Score: 8.94
Episode 2000	Average Score: 9.02
Episode 2000	Average Score: 9.10
Episode 2000	Average Score: 9.18
Episode 2000	Average Score: 9.26
Episode 2000	Average Score: 9.34
Episode 2000	Average Score: 9.42
Episode 2000	Average Score: 9.50
Episode 2000	Average Score: 9.58
Episode 2000	Average Score: 9.66
Episode 2000	Average Score: 9.74
Episode 2000	Average Score: 9.82
Episode 2000	Average Score: 9.90
Episode 2000	Average Score: 9.97
Episode 2000	Average Score: 10.04
Episode 2000	Average Score: 10.11
Episode 2000	Average Score: 10.18
Episode 2000	Average Score: 10.25
Episode 2000	Average Score: 10.32
Episode 2000	Average Score: 10.39
Epis

KeyboardInterrupt: 

In [ ]:

agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = agent.act(state)       # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

In [ ]:
env.close()